# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [42]:
# import libraries

import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import sqlite3
import sqlalchemy
import sys
import io



from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

[nltk_data] Downloading package punkt to /Users/chrismo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/chrismo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
import xgboost as xgb

In [73]:
# Connect to known databases
conn_chrismoetable = sqlite3.connect('chrismoetable.db')
conn_disasterresponse = sqlite3.connect('DisasterResponse.db')
conn_insertdb = sqlite3.connect('InsertDatabaseName.db')

def list_tables(connection):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return connection.execute(query).fetchall()

# List tables for each database
chrismoetable_tables = list_tables(conn_chrismoetable)
disasterresponse_tables = list_tables(conn_disasterresponse)
insertdatabase_tables = list_tables(conn_insertdb)

# Close the connections
conn_chrismoetable.close()
conn_disasterresponse.close()
conn_insertdb.close()

# Output the results
print(chrismoetable_tables)
print(disasterresponse_tables)
print(insertdatabase_tables)

[]
[('Message',)]
[('process_data_table',), ('chrismo',), ('final',)]


In [75]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('final', con=engine)

In [116]:
df.isna()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26025,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26026,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26027,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26028,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [79]:
df.nunique()

related                   2
request                   2
offer                     2
aid_related               2
medical_help              2
medical_products          2
search_and_rescue         2
security                  2
military                  2
child_alone               1
water                     2
food                      2
shelter                   2
clothing                  2
money                     2
missing_people            2
refugees                  2
death                     2
other_aid                 2
infrastructure_related    2
transport                 2
buildings                 2
electricity               2
tools                     2
hospitals                 2
shops                     2
aid_centers               2
other_infrastructure      2
weather_related           2
floods                    2
storm                     2
fire                      2
earthquake                2
cold                      2
other_weather             2
direct_report       

In [81]:
df['related'].value_counts()

related
1.0    19906
0.0     6122
Name: count, dtype: int64

In [84]:
df['request'].value_counts()

request
0.0    21554
1.0     4474
Name: count, dtype: int64

In [86]:
X = df.iloc[:,1]
Y = df.iloc[:,4:]

In [88]:
print(X.shape)
print(Y.shape)

(26028,)
(26028, 32)


In [90]:
Y.isna().sum()

medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report             0
dtype: int64

In [92]:
Y.nunique()

medical_help              2
medical_products          2
search_and_rescue         2
security                  2
military                  2
child_alone               1
water                     2
food                      2
shelter                   2
clothing                  2
money                     2
missing_people            2
refugees                  2
death                     2
other_aid                 2
infrastructure_related    2
transport                 2
buildings                 2
electricity               2
tools                     2
hospitals                 2
shops                     2
aid_centers               2
other_infrastructure      2
weather_related           2
floods                    2
storm                     2
fire                      2
earthquake                2
cold                      2
other_weather             2
direct_report             2
dtype: int64

In [102]:
Y.columns.nunique()

32

In [104]:
df.index += 1

In [106]:
df.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [108]:
X.head(2)

0    0.0
1    0.0
Name: request, dtype: float64

In [110]:
Y.head(2)

,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [114]:
X.value_counts()

request
0.0    21554
1.0     4474
Name: count, dtype: int64

In [36]:
Y.nunique()

related                   2
request                   2
offer                     2
aid_related               2
medical_help              2
medical_products          2
search_and_rescue         2
security                  2
military                  2
child_alone               1
water                     2
food                      2
shelter                   2
clothing                  2
money                     2
missing_people            2
refugees                  2
death                     2
other_aid                 2
infrastructure_related    2
transport                 2
buildings                 2
electricity               2
tools                     2
hospitals                 2
shops                     2
aid_centers               2
other_infrastructure      2
weather_related           2
floods                    2
storm                     2
fire                      2
earthquake                2
cold                      2
other_weather             2
direct_report       

### 2. Write a tokenization function to process your text data

In [39]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [86]:
pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=400, max_depth=32 ,min_samples_split=10, n_jobs=-1, bootstrap=True, random_state=42, verbose=1)))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [89]:
X.shape

(26028,)

In [91]:
Y.shape

(26028, 36)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size = 0.3)

In [95]:
pipeline.fit(X_train, y_train)

/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x16b2472e0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               device=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metr...
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               multi_strategy=None,
                                                               n_estimators=None,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None, ...)))])

In [101]:
y_pred = pipeline.predict(X_test)

In [102]:
acc = accuracy_score(y_test, y_pred)
print(acc)

0.2944038929440389


accuracy = accuracy_score(y_test, y_pred_grid)
precision = precision_score(y_test, y_pred_grid, average='micro')
recall = recall_score(y_test, y_pred_grid, average='micro')


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [107]:
def capture_classification_report(y_true, y_pred):
    # Redirect stdout to capture the print output
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    try:
        print(classification_report(y_true, y_pred))
    finally:
        sys.stdout = old_stdout
    return buffer.getvalue()

# Iterate through each column in the target dataframe Y
for column in Y.columns:
    report = capture_classification_report(y_test[column], y_pred[:, Y.columns.get_loc(column)])
    print(f"Category: {column}\n")
    print(report)
    print("\n" + "="*80 + "\n")

Category: related

              precision    recall  f1-score   support

         0.0       0.70      0.43      0.53      1868
         1.0       0.84      0.94      0.89      5941

    accuracy                           0.82      7809
   macro avg       0.77      0.69      0.71      7809
weighted avg       0.81      0.82      0.80      7809




Category: request

              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94      6476
         1.0       0.78      0.61      0.68      1333

    accuracy                           0.90      7809
   macro avg       0.85      0.79      0.81      7809
weighted avg       0.90      0.90      0.90      7809




Category: offer

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7775
         1.0       0.00      0.00      0.00        34

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg 

### 6. Improve your model
Use grid search to find better parameters. 

In [110]:
parameters = {
                'clf__estimator__n_estimators': [100, 200],
                'clf__estimator__max_depth': [3, 6, 10],
                'clf__estimator__learning_rate': [0.01, 0.1]
    
              }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=1, cv=3)

In [112]:
print(cv)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x16b2472e0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                                                      booster=None,
                                                                                      callbacks=None,
                                                                                      colsample_bylevel=None,
                                                                                      colsample_bynode=None,
                                                                                      colsample_bytree=None,
                                                                               

In [ ]:
# Fit the grid search
cv.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' 

In [ ]:
# Best parameters
best_params = cv.best_params_
#preditcitng using best estimator 
print("Best parameters found: ", best_params)

In [ ]:
y_pred_grid = cv.best_estimator_.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
accuracy = accuracy_score(y_test, y_pred_grid)
precision = precision_score(y_test, y_pred_grid, average='micro')
recall = recall_score(y_test, y_pred_grid, average='micro')


In [ ]:
print(f"Accuracy: {accuracy}")

In [ ]:
print(f"Precision: {precision}")

In [ ]:
print(f"Recall: {recall}")

In [ ]:
print("\nClassification Report:\n", classification_report(y_test, y_pred_grid))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.